In [85]:
import os 
import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [86]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv('../../.env')

True

In [87]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens = 1000)

In [88]:
df_articles = pd.read_csv('../../data/df_articles.csv').dropna()
df_articles['content'] = df_articles['title'] + df_articles['content']

df_articles_analytics = pd.read_csv('../../data/df_articles_analytics.csv').dropna()
df_articles_analytics['content'] = df_articles_analytics['title'] + df_articles_analytics['content']

documents = df_articles['content'].to_list() + df_articles_analytics['content'].to_list()
with open('../../data/courses.txt', 'r') as f:
    documents += list(f.read().split('\n\n\n\n'))

In [89]:
documents = [doc.replace('\xa0', ' ').replace('\n', ' ') for doc in documents]

In [90]:
len(documents)

525

In [91]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(" ".join(documents))

In [ ]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="DeepPavlov/rubert-base-cased-sentence", model_kwargs={'device': 'cpu'})

db = Chroma.from_texts(chunks, embedding_function, persist_directory="../../artifacts/chroma_db")

No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [125]:
import pickle

# Сохраняем chain как pickle
with open("../../artifacts/embedding_function.pkl", "wb") as file:
    pickle.dump(embedding_function, file)

In [93]:
retriever = db.as_retriever()

In [94]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [114]:
from langchain.load import dumps, loads, load

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [96]:
question = "Какие акции IT компаний сейчас наиболее интересны для приобретения?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

7

In [97]:
docs

[Document(page_content='поехали! В этом уроке мы дадим вам универсальный чек-лист. С ним вы в любой момент быстро оцените ситуацию вокруг и подберете себе актив. ☝🏻 Хорошая новость: чек-лист подойдет не только для выбора акций, но и если вы захотите присмотреться к облигациям или фондам. Вообще, когда мы говорим про выбор активов, принцип практически одинаковый. Поэтому мы и назвали чек-лист универсальным 🧐 Ну что, поехали! 1. Новости Проверить финансовые показатели компании перед покупкой — дело важное, но не единственное. Инвестор должен быть в курсе событий, связанных с компанией, чтобы не пропустить что-то важное (аварии, форс-мажоры, слияние, выплату дивидендов и прочее). 🤔 Зачем смотреть Слухи и новости о компании, высказывания первых лиц или же другая информация, имеющая прямое или косвенное отношение к бизнесу компании, могут повлиять на котировки. 1. Новости Проверить финансовые показатели компании перед покупкой — дело важное, но'),
 Document(page_content='перспективе за счет

In [98]:
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

res = final_rag_chain.invoke({"question":question})

print(res)

На основании предоставленного контекста, можно выделить несколько факторов, которые делают акции IT-компаний интересными для приобретения:

1. **Развитие собственных продуктов**: Упоминается, что компании, развивающие собственные решения, показывают высокую рентабельность. Например, валовая прибыль от собственных продуктов составляет около 60%, что значительно выше, чем у сторонних продуктов (около 13%). Это делает такие компании более привлекательными для инвестиций.

2. **Структурные изменения на рынке**: Компании, которые выигрывают от перестройки IT-рынка и активного импортозамещения, также могут быть интересными для инвесторов. Это связано с ростом прибыльности и возможностью захвата новых рыночных ниш.

3. **Слияния и поглощения (M&A)**: Компании, активно занимающиеся сделками слияний и поглощений, могут продемонстрировать значительный рост. Например, одна из компаний завершила более восьми сделок в разных сегментах рынка, что может свидетельствовать о её стратегическом развитии 

In [107]:
from langchain_core.load.serializable import to_json_not_implemented

repr = to_json_not_implemented(final_rag_chain)

In [108]:
repr

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain_core', 'runnables', 'base', 'RunnableSequence'],
 'repr': "{\n  context: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \\ndifferent versions of the given user question to retrieve relevant documents from a vector \\ndatabase. By generating multiple perspectives on the user question, your goal is to help\\nthe user overcome some of the limitations of the distance-based similarity search. \\nProvide these alternative questions separated by newlines. Original question: {question}'))])\n           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x308f7a490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3424bcb90>, root_client=<openai.OpenAI object at 0x177459690>, root_async_client=<openai.AsyncOpenAI obj

In [104]:
string_representation = dumps(final_rag_chain, pretty=True)
print(string_representation[:500])

{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "vectorstores",
    "base",
    "VectorStoreRetriever"
  ],
  "repr": "VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x111af9090>)",
  "name": "VectorStoreRetriever"
}


In [109]:
import json

with open("../../artifacts/chain.json", "w") as fp:
    json.dump(repr, fp)

In [120]:
with open("../../artifacts/chain.json", "r") as fp:
    chain_dict = json.load(fp)

In [121]:
chain_dict

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain_core', 'runnables', 'base', 'RunnableSequence'],
 'repr': "{\n  context: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \\ndifferent versions of the given user question to retrieve relevant documents from a vector \\ndatabase. By generating multiple perspectives on the user question, your goal is to help\\nthe user overcome some of the limitations of the distance-based similarity search. \\nProvide these alternative questions separated by newlines. Original question: {question}'))])\n           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x308f7a490>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3424bcb90>, root_client=<openai.OpenAI object at 0x177459690>, root_async_client=<openai.AsyncOpenAI obj